1) Xác minh nhanh dataset (count / sanity check)
Đếm số file theo folder (biết ngay class balance):

In [ ]:
import os, collections
root = "Dataset"
cnt = collections.Counter()
for dirpath, _, files in os.walk(root):
    for f in files:
        if f.lower().endswith(".exe"):
            # parent label: giả sử folder 1 cấp trên là label (Benign hoặc Virus)
            label = os.path.basename(os.path.dirname(dirpath)) if os.path.basename(dirpath) else os.path.basename(dirpath)
            cnt[label] += 1
print(cnt)

2) Kiểm tra file có phải PE hợp lệ (sanity)
Skip files không phải PE (dùng pefile):

In [ ]:
import pefile, os
bad=[]
for root,_,files in os.walk("Dataset"):
  for f in files:
    if f.lower().endswith(".exe"):
      p=os.path.join(root,f)
      try:
        pefile.PE(p)
      except Exception:
        bad.append(p)
print("Bad files:", bad)


In [ ]:
python bytes_to_png_batch.py --input_dir Dataset --out_dir converted --width 256

Kiểm tra 1 ảnh và reconstruct thử:

In [ ]:
python png_back_to_exe.py --png converted/images/Benign/sample.exe.png --meta converted/meta/Benign/sample.exe.json --out restored
# so sánh file gốc và restored bằng sha256 hoặc cmp
sha256sum Dataset/Benign/sample.exe restored/sample.exe


Nếu khớp => reversible OK.

4) Trích xuất features dạng bảng (header, sections, dlls, entropy, opcodes nếu có)

Chạy merge_features.py (script đã chuẩn bị) để tạo merged_features.csv:


In [ ]:
python merge_features.py --input_dir Dataset --out_csv merged_features.csv

Nếu bạn chưa cài capstone để lấy opcodes, cài trước hoặc chạy với option skip opcode (mình có thể cập nhật script để skip tự động nếu thiếu capstone).

Cần chú ý: bước này có thể lâu (disassembly).

5) EDA (đọc CSV + images)

Mở notebook (mình sẽ cung cấp) hoặc chạy nhanh:

Kiểm tra balance:


In [ ]:
import pandas as pd
df = pd.read_csv("merged_features.csv")
print(df["label"].value_counts())

Plot file size / entropy distribution:

In [ ]:

import seaborn as sns
sns.histplot(df, x="file_size", hue="label", log_scale=True)
sns.histplot(df, x="file_entropy", hue="label", bins=50)

PCA t-SNE trên một tập con numeric features (header + opcodes) để xem phân tách.

Mục tiêu: phát hiện imbalance / outliers / packed files (entropy cao).